In [1]:
import requests
# https://stackoverflow.com/questions/24346872/python-equivalent-of-a-given-wget-command

r = requests.get("https://haltools.inria.fr/Public/exportPubli.php?format_export=bibtex&annee_publideb=2014&labos_exp=diverse;triskell&CB_auteur=oui&CB_titre=oui&CB_article=oui&langue=Anglais&tri_exp=typdoc&tri_exp2=auteur_exp&tri_exp3=titre&ordre_aff=TA")
bibname_diverse = "diverse.bib"
open(bibname_diverse , 'wb').write(r.content)

290538

In [3]:
import bibtexparser


with open(bibname_diverse) as bibtex_file:
    # bib_database = bibtexparser.load(bibtex_file)
    bib_database = bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(bibtex_file)
len(bib_database.entries)

432

In [4]:
import xml.etree.ElementTree as ET
import pandas as pd

def get_abstract(hal_id):
    url = "https://hal.inria.fr/" + hal_id + "/metadata/?wt=json"
    r = requests.get(url)
    root = ET.fromstring(r.content)
    for child in root.iter('{http://www.tei-c.org/ns/1.0}abstract'):
        return child.find('{http://www.tei-c.org/ns/1.0}p').text
    return None
    
# print(get_abstract('hal-01688247'))
df = pd.DataFrame(columns=['hal_id', 'year', 'title', 'authors', 'abstract'])
i = 0
for entry in bib_database.entries:
    # if 'year' in entry:
    year = int(entry['year'])
    authors = entry['author']
    # if (int(year) >= 2014):
    bkey = entry['ID'] # can retrieved hal_id out of key
    hal_id = entry['hal_id']
    title = entry['title']

    abstract = get_abstract(hal_id)
    # process
    # extract hal-id first based on key or hal_id?
    df.loc[i] = (hal_id, year, title, authors, abstract)
    i = i + 1
                # https://hal.inria.fr/hal-00746566/metadata
        

In [5]:
df

,hal_id,year,title,authors,abstract
0,hal-03897639,2023,{BURST: Benchmarking Uniform Random Sampling T...,"Acher, Mathieu and Perrouin, Gilles and Cordy,...",BURST is a benchmarking platform for uniform r...
1,hal-01522779,2017,{Teaching Software Product Lines: A Snapshot o...,"Acher, Mathieu and Lopez-Herrejon, Roberto Eri...",Software Product Line (SPL) engineering has em...
2,hal-01088464,2014,{Consistency Checking in Early Software Produc...,"Alférez, Mauricio and Lopez-Herrejon, Roberto ...",Software Product Line Engineering (SPLE) is a ...
3,hal-01088537,2014,{Evaluating scenario-based SPL requirements ap...,"Alférez, Mauricio and Bonifácio, Rodrigo and T...",Software product lines (SPL) provide support f...
4,hal-01688247,2019,{Modeling Variability in the Video Domain: Lan...,"Alférez, Mauricio and Acher, Mathieu and Galin...","[Context] In an industrial project, we address..."
...,...,...,...,...,...
427,hal-03793085,2022,{The Indiscernibility Methodology: quantifying...,"Marquer, Yoann and Zendra, Olivier and Heuser,...",Cyber security threats are important and growi...
428,hal-01634288,2017,{The Emerging Field of Test Amplification: A S...,"Monperrus, Martin and Danglot, Benjamin and Ve...",Context: The increasing adoption of test-drive...
429,hal-04130361,2023,{PyroBuildS: Enabling Efficient Exploration of...,"Randrianaina, Georges Aaron and Khelladi, Djam...",Software engineers are acutely aware that the ...
430,hal-01401828,2016,{Approximate Loop Unrolling},"Rodriguez-Cancio, Marcelino and Combemale, Ben...","We introduce Approximate Unrolling, a loop opt..."


In [32]:
import time
import random

def get_summary(abstract):
    if abstract is None:
        return None
    headers = {
        'Content-Type': 'application/json',
    }
    data = '{"question":' + '"' + abstract + '"' + ',' + '"choices":["",""]}'
    response = requests.post('https://scitldr.apps.allenai.org/api/solve', headers=headers, data=data.encode('utf8'))
    esummarization = response.json()['answer']
    return esummarization

df['esummary'] = None
for k, v in df.query("year >= 2014").iterrows():
    abstract = v['abstract']
    if abstract is not None:
        abstract = abstract.replace('{', '').replace('}', '').replace('“', "").replace("\'", "").replace('\~', ' ').replace('\&', '&').replace('"', "").replace('\%', '%').rstrip().lstrip().replace('\n', '')
        esummary = get_summary(abstract)
        df.loc[k, 'esummary'] = esummary
        print(esummary)
    time.sleep(random.randint(2, 10))
df.query("year >= 2014")

In this article we report on three initiatives we have conducted with scholars, educators, industry practitioners, and students to further understand the connection between the connection
Software Product Line Engineering (SPLE) is a successful paradigm to produce a family of products for a specific domain.
This paper presents a metric-based evaluation aiming at assessing how well the chosen qualities are supported by scenario-based SPL requirements approaches.
VM: Software Variance Modeling and Control in the Video Domain .
Software diversification beyond the operating system level: a realistic Internet application .
This paper proposes to explore the question of the formal verification of model transformation properties through a tridimensional approach.
Model transformation traceability in conjunction with a model of mutation operators, and a dedicated algorithm for the test data set improvement activity .
Si au lieu of distribuer des millions de copies identiques, les fabricants de

We propose a new middleware dedicated to CPS to enable the management of software deployment and the dynamic reconfiguration of these systems.
This work discusses relevant challenges and requirements for integrating cloud computing with pervasive systems operating in dynamic environments.
The Dagstuhl Manifesto is a manifesto which places emphasis on the scholarly value of academic software and on personal responsibility.
In this paper, we report on preliminary experiments showing the capability of state-of-the-art classification algorithms to assist the configuration process.
This tutorial provides a practical approach for developing and integrating various Domain-Specific (modeling) Languages (DSLs) used in the development of
A low cost and efficient video transcoding platform over an ad hoc computing cloud built around a rack of Raspberry Pis.
DIVERSIFY is an EU funded project, which aims at favoring spontaneous diversification in software systems in order to increase their adaptive

A new domain specific language that leverages Family Polymorphism to allow model polymorphism, inheritance among DSLs, as well as the safe reuse
Towards Safe Reuse of Model Transformations in Domain Specific Languages: A Research Roadmap .
Challenges in Designing Domain Specific Languages: A Research Roadmap for addressing them .
We present an exploratory approach to enable the expression of whole model transformations in OCL.
This work presents a collaboration framework for collaborating groups capable of providing synchronous and asynchronous mode of collaboration.
Coq4MDE: A Coq proof assistant for language specific composition and verification technologies .
This paper presents evidence that a canonical set of reuse interfaces has emerged over time: the variation, customization, and usage interfaces (VCU)
This paper presents a formal analysis framework to analyze a family of platform products w.r.t. real-time properties.
We show that browser fingerprinting is as effective on mobile

A model slicing technique to improve the scalability of existing invariant checking tools.
This paper proposes a new method, called Multimorphic testing, to assess the relative effectiveness of a test suite for revealing performance variations of a
Software product line (SPL) engineers put a lot of effort to ensure that, through the setting of a large number of possible configuration options,
We propose to use machine learning to infer product-line constraints from an oracle that is able to assess whether a given product is correct.
KevoreeJS: A component-based platform to control browser as component platform for single-page web page applications .
A new approach to automatically infer a relevant variability model from a collection of already implemented language components, given a general representation of the domain.
This paper discusses the merits and challenges of adopting software product line engineering (SPLE) as the main development process for an automotive Hall Effect senso

ConnectionError: HTTPSConnectionPool(host='scitldr.apps.allenai.org', port=443): Max retries exceeded with url: /api/solve (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fc7042aa3d0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [33]:
# df.query("year > 2019")['esummary']
df.to_csv('diverse-esummary.csv')

In [35]:
# df.loc[5]['abstract'] # shity characters

In [6]:
df.to_csv('diverse_summary.csv')